In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import category_encoders as ce

import geopy

In [ ]:
# загружаем данные 
DATA_DIR = '/kaggle/input/sf-booking/'
hotels_train = pd.read_csv(DATA_DIR + 'hotels_train.csv')
hotels_test = pd.read_csv(DATA_DIR + 'hotels_test.csv')
sample_submission = pd.read_csv(DATA_DIR + '/submission.csv')

In [ ]:
hotels_train[:2]

In [ ]:
# определяем долю рецензентов по национальному признаку
reviewer_nationality = hotels_train['reviewer_nationality'].value_counts(normalize=True)

# подавляющее большинство рецензий (47,5 %) написано рецензентами из United Kingdom, доля остальных существенно ниже
# оставим в датасете названия только тех стран, доля рецензентов из которых больше 1 %, остальные страны отнесем в категорию Other_nationalitity 

# определим страны с наибольшим (>= 1%) рецензентов
countries = reviewer_nationality[reviewer_nationality >= 0.01].index
hotels_train['reviewer_nationality'] = hotels_train['reviewer_nationality'].apply(lambda x: x.strip() if x in countries else 'Other_nationalities')
sns.histplot(hotels_train['reviewer_nationality'])
plt.xticks(rotation=90);

In [ ]:
# преобразуем даты в формат datetime
hotels_train['review_date'] = pd.to_datetime(hotels_train['review_date'])

In [ ]:
# создаем признаки года и месяца
hotels_train['year'] = pd.to_datetime(hotels_train['review_date']).dt.year
hotels_train['month'] = pd.to_datetime(hotels_train['review_date']).dt.month

In [ ]:
# создаем признаков на основе положительных отзывов
import re
hotels_train['location_positive'] = hotels_train['positive_review'].apply(lambda x: 1 if re.findall(r'location', x.lower()) else 0)
hotels_train['breakfast_positive'] = hotels_train['positive_review'].apply(lambda x: 1 if re.findall(r'breakfast', x.lower()) else 0)
hotels_train['staff_positive'] = hotels_train['positive_review'].apply(lambda x: 1 if re.findall(r'staff', x.lower()) else 0)
hotels_train['everything_positive'] = hotels_train['positive_review'].apply(lambda x: 1 if re.findall(r'everything|all', x.lower()) else 0)
hotels_train['nothing_positive'] = hotels_train['positive_review'].apply(lambda x: 1 if re.findall(r'nothing', x) else 0)

In [ ]:
# создание признаков на основе отрицательных отзывов
import re
hotels_train['nothing_negative'] = hotels_train['negative_review'].apply(lambda x: 1 if re.findall(r'nothing|none|no complaints|nil|all good| great| perfect', x.lower()) else 0)
hotels_train['everything_negative'] = hotels_train['negative_review'].apply(lambda x: 1 if re.findall(r'everything', x.lower()) else 0)
hotels_train['price_negative'] = hotels_train['negative_review'].apply(lambda x: 1 if re.findall(r'price|expensive', x.lower()) else 0)
hotels_train['location_negative'] = hotels_train['negative_review'].apply(lambda x: 1 if re.findall(r'location', x.lower()) else 0)
hotels_train['staff_negative'] = hotels_train['negative_review'].apply(lambda x: 1 if re.findall(r'staff', x.lower()) else 0)
hotels_train['room_negative'] = hotels_train['negative_review'].apply(lambda x: 1 if re.findall(r'room', x.lower()) else 0)
hotels_train['breakfast_negative'] = hotels_train['negative_review'].apply(lambda x: 1 if re.findall(r'breakfast', x.lower()) else 0)

In [ ]:
# удаляем признаки, из которых созданы новые признаки
hotels_train = hotels_train.drop(['positive_review', 'negative_review'], axis=1)

In [ ]:
# функция для извлечения первого тега
def get_first_tag(x):
    tags = eval(x)
    if len(tags) >= 1 and tags[0].strip().endswith('trip'):
        return tags[0].strip()
    else:
        return 'unknown'

# создаем признак  тип поездки
hotels_train['trip_type'] = hotels_train['tags'].apply(get_first_tag)
hotels_train['trip_type'].value_counts()

In [ ]:
# функция для извлечения второго тега
def get_second_tag(x):
    tags = eval(x)
    if len(tags) >= 2:
        for tag in tags:
            if tag in [' Couple ', ' Solo traveler ', ' Group ', ' Family with older children ', ' Family with young children ', ' Travelers with friends ', ' With a pet ']:
                return tag.strip()
    else:
        return 'unknown'

# создаем признак типа гостей
hotels_train['guests'] = hotels_train['tags'].apply(get_second_tag)
hotels_train['guests'].value_counts()

In [ ]:
hotels_train['guests'].value_counts()
# # строк с 'unknown' всего 5, в других признаках также мало информации, лучше эти строки удалить
# hotels_train[hotels_train['guests'] == 'unknown'].index
# hotels_train = hotels_train.drop(list(hotels_train[hotels_train['guests'] == 'unknown'].index), axis=0)

In [ ]:
# функция для извлечения третьего тега
import re
def get_third_tag(x):
    # elements = re.split(r',', x)
    # if len(elements) >= 3:
    # for element in elements:
    if re.findall(r'Double|Twin|2 rooms', x):
        return 'Double'
    elif re.findall(r'Triple', x):
        return 'Triple'
    elif re.findall(r'Single|Suite|Room |Guestroom ', x):
        return 'Single'
    elif re.findall(r'Studio', x):
        return 'Studio'
    else:
        return 'unknown'
    
# создаем признак типа номера по количеству комнат
hotels_train['apartment'] = hotels_train['tags'].apply(get_third_tag)
hotels_train['apartment'].value_counts()

In [ ]:
# функция для извлечения четвертого тега
def get_fourth_tag(x):
    tags = eval(x)
    if len(tags) > 3:
        tags = tags[3].split()
        for tag in tags:
            if tag.isdigit(): 
                return int(tag)
    else:
        return np.nan
    
# создаем признак количества ночей
hotels_train['count_nights'] = hotels_train['tags'].apply(get_fourth_tag)
hotels_train['count_nights']

In [ ]:
# проверим сколько пропущенных значений в признаке количества ночей
hotels_train['count_nights'].isna().sum()

In [ ]:
# заполним пропуски медианным значением
hotels_train['count_nights'].fillna(value=hotels_train['count_nights'].median(), inplace=True)
sns.histplot(hotels_train['count_nights'],
             bins=30);

In [ ]:
# проверяем, что не осталось пропущенных значений
hotels_train['count_nights'].isna().sum()

In [ ]:
hotels_train['count_nights'].value_counts(normalize=True) * 100

In [ ]:
# функция для извлечения количества дней с даты написания рецензии
def get_nums(x):
    x = x.split()
    if x[0].isdigit():
        return int(x[0])
    
# создание числового признака количества дней
hotels_train['days_since_review'] = hotels_train['days_since_review'].apply(get_nums)

### кодирование категориальных признаков с помощью метода OneHotEncoding

In [ ]:
# кодируем признак типа поездки
encoder = ce.OneHotEncoder(cols=['trip_type'], use_cat_names=True)
df = encoder.fit_transform(hotels_train['trip_type'])
hotels_train = pd.concat([hotels_train, df], axis=1)

# кодируем признак типа гостей
encoder = ce.OneHotEncoder(cols=['guests'], use_cat_names=True)
df = encoder.fit_transform(hotels_train['guests'])
hotels_train = pd.concat([hotels_train, df], axis=1)

# кодирование признака типа номера
enc = ce.OneHotEncoder(hotels_train['apartment'], use_cat_names = True)
df = enc.fit_transform(hotels_train['apartment'])
hotels_train = pd.concat([hotels_train, df], axis=1)

# кодирование признака национальности
nationality = pd.get_dummies(hotels_train['reviewer_nationality'])
hotels_train = pd.concat([hotels_train, nationality], axis=1)
hotels_train = hotels_train.drop(['reviewer_nationality'], axis=1)

### удаление текстовых признаков

In [ ]:
# удаляем признаки типов гостей, номеров, дату обзора и столбец с тегами 
hotels_train = hotels_train.drop(['guests', 'trip_type', 'apartment', 'review_date', 'tags'], axis=1)

hotels_train['hotel_name'].value_counts().shape
# удаляем признак названия отеля как малоинформативный
hotels_train = hotels_train.drop(['hotel_name'], axis=1)

In [ ]:
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_timeout = 7

# функция для извленчения широты
def get_lng(x):
    geolocator = Nominatim(user_agent='spazz')
    address = ' '.join(x.split()[-4:])
    location = geolocator.geocode(address)
    if location is not None:
        return location.longitude
    else:
        return None
    
# функция для извленчения долготы
def get_lat(x):
    geolocator = Nominatim(user_agent='spazz')
    address = ' '.join(x.split()[-4:])
    location = geolocator.geocode(address)
    if location is not None:
        return location.latitude
    else:
        return None

In [ ]:
# заменяем в столбцах пропущенные значения широты 
longitudes = hotels_train['hotel_address'][hotels_train['lng'].isna()].apply(get_lng)
indexes = longitudes.index
hotels_train.loc[indexes, 'lng'] = longitudes

In [ ]:
# и долготы
latitudes = hotels_train['hotel_address'][hotels_train['lat'].isna()].apply(get_lat)
indexes = latitudes.index
hotels_train.loc[indexes, 'lat'] = latitudes

In [ ]:
# проверяем, что пропущенные значения отсутствует
hotels_train['lat'].isnull().sum(), hotels_train['lng'].isnull().sum()

In [ ]:
# удаляем текстовый признак адреса отеля
hotels_train = hotels_train.drop(['hotel_address'], axis=1)

In [ ]:
# проверяем на наличие дубликатов
display(hotels_train.duplicated().sum())
# удаляем дублирующиеся строки
# hotels_train.drop_duplicates(inplace=True)

In [ ]:
# удаляем неинформативные признаки
hotels_train = hotels_train.drop(['trip_type_unknown'], axis=1)
hotels_train = hotels_train.drop(['apartment_unknown'], axis=1)

In [ ]:
# разделяем признаки на непрерывные и категориальные
num_cols = ['additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts', 
            'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 
            'days_since_review']
cat_cols = [col for col in hotels_train.columns if col not in num_cols and col != 'reviewer_score']

In [ ]:
cat_cols.remove('lng')
cat_cols.remove('lat')

In [ ]:
# для оценки значимости категориальных переменных используем критерий хи-квадрат
X = hotels_train[cat_cols]
y = hotels_train['reviewer_score']

y = y.astype('int')
from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
fig, ax = plt.subplots(figsize=(8, 8))
imp_cat.plot(kind = 'barh', ax=ax);

In [ ]:
# для оценки значимости непрерывных числовых переменных используем дисперсионный анализ
X = hotels_train[num_cols]  
y = hotels_train['reviewer_score']

from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels_train[num_cols]  
y = hotels_train['reviewer_score']

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels_train.drop(['reviewer_score'], axis = 1)  
y = hotels_train['reviewer_score']

In [ ]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)